In [2]:
!ls
import json

Problem8Attempts.json Process Data.ipynb    attempts.txt


In [33]:
def answer_should_be_int(att_eval_tree):
    if len(att_eval_tree)==1:
        final_value=att_eval_tree[0]
    elif len(att_eval_tree)>1:
        final_value=att_eval_tree[0][1]
    else:
        final_value=0
    if int(final_value)!=final_value:
        return "Can the answer to this question be a fractional number?(y,n)  [__]{'n'}"
    else:
        return ""
        
def answer_is_not_expression(att_eval_tree):
    if len(att_eval_tree)==1:
        return "Please enter an expression, not just final numerical result"
    else:
        return ""

In [34]:
def check_81(params):
    attempt,att_parse_tree,att_eval_tree,d1,d2,d3,variables = params

    hint=answer_should_be_int(att_eval_tree)
    if  len(hint)!=0: 
        return hint
    
    hint=answer_is_not_expression(att_eval_tree)
    if  len(hint)!=0: 
        return hint

    return None

In [36]:
file=open('Problem8Attempts.json','r');
answer=json.loads(file.readline())['answer']
print 'answer=',answer
for line in file.readlines():
    params=json.loads(line);
    attempt,att_parse_tree,att_eval_tree,d1,d2,d3,variables = params
    hint=check_81(params)
    
    #if hint !=None:
    #    print attempt,hint
    
    if hint==None:
        print attempt,answer,variables

answer= 26**$n2-25**$n2
6^2-5^2 26**$n2-25**$n2 {u'$n2': 5.0}
6*2-1 26**$n2-25**$n2 {u'$n2': 6.0}
6^2-5^2 26**$n2-25**$n2 {u'$n2': 3.0}
6^2-5^2 26**$n2-25**$n2 {u'$n2': 3.0}
3*26*26 26**$n2-25**$n2 {u'$n2': 3.0}
(26*26*26) - (25*25*25) 26**$n2-25**$n2 {u'$n2': 3.0}
2! 26**$n2-25**$n2 {u'$n2': 6.0}
6! 26**$n2-25**$n2 {u'$n2': 6.0}
6!/2! 26**$n2-25**$n2 {u'$n2': 6.0}
6+6+1 26**$n2-25**$n2 {u'$n2': 3.0}
5+5+1 26**$n2-25**$n2 {u'$n2': 3.0}
6^2-3 26**$n2-25**$n2 {u'$n2': 4.0}
6^2 26**$n2-25**$n2 {u'$n2': 4.0}
6^2-6^1 26**$n2-25**$n2 {u'$n2': 4.0}
6^2-5^2 26**$n2-25**$n2 {u'$n2': 4.0}
36-12 26**$n2-25**$n2 {u'$n2': 5.0}
30+30+1 26**$n2-25**$n2 {u'$n2': 6.0}
C(6,2) 26**$n2-25**$n2 {u'$n2': 3.0}
36 - 2 26**$n2-25**$n2 {u'$n2': 3.0}
36 - 3 26**$n2-25**$n2 {u'$n2': 3.0}
36 - 4 26**$n2-25**$n2 {u'$n2': 3.0}
36-5 26**$n2-25**$n2 {u'$n2': 3.0}
36-6 26**$n2-25**$n2 {u'$n2': 3.0}
36-7 26**$n2-25**$n2 {u'$n2': 3.0}
36-8 26**$n2-25**$n2 {u'$n2': 3.0}
36-16 26**$n2-25**$n2 {u'$n2': 3.0}
36-10 26**$n2-25